In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk, ImageFilter, ImageOps
import cv2
import numpy as np

file_path = None  # Global variable to store the file path
sketch_img_pil = None  # Global variable to store the sketch image in PIL format

def convert_to_sketch(original_img):
    try:
        # Convert the image to grayscale
        gray_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
        # Invert the grayscale image
        inverted_gray_img = 255 - gray_img
        # Blur the inverted grayscale image
        blurred_img = cv2.GaussianBlur(inverted_gray_img, (21, 21), 0)
        # Invert the blurred image
        inverted_blurred_img = 255 - blurred_img
        # Blend the inverted blurred image with the grayscale image using Color Dodge
        pencil_sketch_img = cv2.divide(gray_img, inverted_blurred_img, scale=256.0)
        return pencil_sketch_img
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {str(e)}")
        return None

def open_file():
    global file_path
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png;*.bmp;*.gif")])
    if file_path:
        original_img = cv2.imread(file_path)
        if original_img is not None:
            display_original_image(original_img)

def display_original_image(original_img):
    # Resize the image to fit in the canvas
    original_img = cv2.resize(original_img, (200, 200))
    # Convert image to PIL format
    original_img_pil = Image.fromarray(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
    # Convert image to Tkinter format
    original_img_tk = ImageTk.PhotoImage(original_img_pil)
    # Display original image on canvas
    canvas.create_image(0, 0, anchor=tk.NW, image=original_img_tk)
    # Save reference to avoid garbage collection
    canvas.original_img = original_img_tk

def create_sketch():
    global file_path, sketch_img_pil
    # Check if an image has been opened
    if file_path:
        original_img = cv2.imread(file_path)
        # Convert the original image to sketch
        sketch_img = convert_to_sketch(original_img)
        if sketch_img is not None:
            # Convert sketch image to PIL format
            sketch_img_pil = Image.fromarray(sketch_img)
            # Resize sketch image to fit canvas
            sketch_img_pil = sketch_img_pil.resize((200, 200))
            # Convert sketch image to Tkinter format
            sketch_img_tk = ImageTk.PhotoImage(sketch_img_pil)
            # Display sketch image on canvas
            canvas.create_image(210, 0, anchor=tk.NW, image=sketch_img_tk)
            # Save reference to avoid garbage collection
            canvas.sketch_img = sketch_img_tk
        else:
            messagebox.showinfo("Info", "Please open an image first.")

def save_sketch():
    global sketch_img_pil
    if sketch_img_pil:
        save_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png"), ("All files", "*.*")])
        if save_path:
            sketch_img_pil.save(save_path)
            messagebox.showinfo("Success", "Sketch saved successfully!")
    else:
        messagebox.showinfo("Info", "No sketch available to save. Please create a sketch first.")

root = tk.Tk()
root.title("Image to Pencil Sketch Converter")

# Create a label for the title of the project
title_label = tk.Label(root, text="Create Pencil Sketch Art", fg="dark orange", font=("Snap ITC", 20, "bold"))
title_label.pack()

# Create a label for selecting an image
label = tk.Label(root, text="Select an image from folder:")
label.pack()

# Create buttons
open_button = tk.Button(root, text="Open Image", command=open_file)
open_button.pack(pady=10)

sketch_button = tk.Button(root, text="Create Sketch", command=create_sketch)
sketch_button.pack(pady=10)

save_button = tk.Button(root, text="Save Sketch", command=save_sketch)
save_button.pack(pady=10)

# Create canvas for displaying images
canvas = tk.Canvas(root, width=420, height=200)
canvas.pack()

root.mainloop()
